In [127]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import math
import re

In [132]:
class Match:
    def __init__(self, home_team=None, away_team=None, home_win=None,
                 draw=None, away_win=None, victory=None):
        self.home_team = home_team
        self.away_team = away_team
        self.home_win = home_win
        self.draw = draw
        self.away_win = away_win
        self.victory = victory
        self.match_page = None
        self.betting_odds = None
        self.home_odds = None
        self.away_odds = None
        self.draw_odds = None

    def get_upcoming_betting_odds(self, username=None, password=None):
        driver = webdriver.Chrome()

        driver.get("https://live.efortuna.pl/")

        driver.maximize_window()
        time.sleep(5)

        cookies_button = driver.find_element(By.ID, "cookie-consent-button-accept")
        cookies_button.click()

        login_button = driver.find_element(By.CSS_SELECTOR, "div.login_form__button")
        login_button.click()

        login_field = driver.find_element(By.NAME, "username")
        password_field = driver.find_element(By.NAME, "password")

        login_field.send_keys(username)
        password_field.send_keys(password)

        # final_login_button = driver.find_element(By.CSS_SELECTOR, "button.button.button-yellow.user-box-form-button")
        # final_login_button.click()

        expand_matches = driver.find_element(By.CLASS_NAME, "sport-header__button")
        expand_matches.click()

        actual_matches = driver.find_elements(By.CLASS_NAME, "live-match-info__team--1")
        for actual_match in actual_matches:
            if actual_match.get_attribute("innerHTML").__contains__(self.home_team):
                match_button = actual_match.find_element(By.XPATH, "./..")
                self.match_page = match_button.get_attribute("href")

        driver.close()

    # date check has to be implemented to be sure it's the correct match
    def get_previous_betting_odds(self):
        driver = webdriver.Chrome()

        driver.get("https://www.flashscore.com/")

        driver.maximize_window()

        cookies_button = driver.find_element(By.ID, "onetrust-reject-all-handler")
        cookies_button.click()

        search_button = driver.find_element(By.CSS_SELECTOR, "#search-window")
        search_button.click()

        input_field = driver.find_element(By.CSS_SELECTOR, "#search-window > div > div > div.searchInput > input")
        input_field.send_keys(self.home_team)

        time.sleep(2)

        home_team_button = driver.find_element(By.CSS_SELECTOR, "#search-window > div > div > div.searchResults > div > a:nth-child(1)")
        home_team_button.click()

        time.sleep(2)

        results_button = driver.find_element(By.CSS_SELECTOR, "#li2")
        results_button.click()

        matches = driver.find_elements(By.CLASS_NAME, "event__participant--away")
        win_before = driver.window_handles[0]
        for m in matches:
            if m.get_attribute("innerHTML").__contains__(self.away_team):
                m.click()
                win_after = driver.window_handles[1]
                driver.switch_to.window(win_after)
                break

        pages = driver.find_elements(By.CLASS_NAME, "tabs__tab")
        for p in pages:
            if p.get_attribute("innerHTML").__contains__("Odds"):
                p.click()
                break

        time.sleep(2)

        self.home_odds = float(driver.find_element(By.CSS_SELECTOR, "div.oddsTab__tableWrapper > div > div.ui-table__body > div:nth-child(1) > a:nth-child(2) > span").get_attribute("innerHTML"))
        self.draw_odds = float(driver.find_element(By.CSS_SELECTOR, "div.oddsTab__tableWrapper > div > div.ui-table__body > div:nth-child(1) > a:nth-child(3) > span").get_attribute("innerHTML"))
        self.away_odds = float(driver.find_element(By.CSS_SELECTOR, "div.oddsTab__tableWrapper > div > div.ui-table__body > div:nth-child(1) > a:nth-child(4) > span").get_attribute("innerHTML"))

        driver.close()
        driver.switch_to.window(win_before)
        driver.close()

In [134]:
match = Match(home_team="Switzerland", away_team='Cameroon')
match.get_previous_betting_odds()
print(match.home_odds)
print(match.draw_odds)
print(match.away_odds)
# username = input("Username: ")
# password = input("Password: ")
# match.get_upcoming_betting_odds(username, password)
# match.get_upcoming_betting_odds('username', 'password')

1.82
3.55
5.0


In [124]:
def get_past_results(matches_number=8, every=False):
    # op = webdriver.ChromeOptions()
    # op.add_argument("--headless")
    driver = webdriver.Chrome()

    driver.get("https://kickoff.ai/matches")

    driver.maximize_window()

    click_number = math.ceil(matches_number/8)
    for i in range(click_number):
        button = driver.find_element(By.CLASS_NAME, "more-matches-all")
        button.click()
        time.sleep(1)

    predictions = driver.find_elements(By.CLASS_NAME, "prediction-result")

    predictions = predictions[:matches_number]
    matches_arr = []
    for prediction in predictions:
        home_team = prediction.find_element(By.CSS_SELECTOR, '.team-home .team-name').get_attribute('innerHTML')
        # home_team = ts.google(home_team, from_language='en', to_language='pl')

        away_team = prediction.find_element(By.CSS_SELECTOR, '.team-away .team-name').get_attribute('innerHTML')
        # away_team = ts.google(away_team, from_language='en', to_language='pl')

        victory = "Draw"

        result = prediction.find_element(By.CLASS_NAME, "result").get_attribute("innerHTML")
        result_arr = re.findall(r'\d+', result)

        if int(result_arr[0]) > int(result_arr[1]):
            victory = home_team
        elif int(result_arr[0]) < int(result_arr[1]):
            victory = away_team

        bars = prediction.find_elements(By.CLASS_NAME, "progress-bar")
        temp_width_arr = []
        temp_sum = 0
        for bar in bars:
            temp_width_arr.append(bar.size['width'])
            temp_sum += bar.size['width']
        for i in range(len(temp_width_arr)):
            temp_width_arr[i] = round((temp_width_arr[i] / temp_sum) * 100)


        match = Match(home_team, away_team, temp_width_arr[0], temp_width_arr[1], temp_width_arr[2], victory)
        match.get_previous_betting_odds()

        matches_arr.append(match)

    driver.close()
    return matches_arr

In [137]:
def count_profit(bet=5, base_percentage=50, tax=0.12, draw_win_odds=1.3):
    matches = get_past_results(10)

    right_call = 0
    wrong_call = 0
    profit = 0


    for m in matches:
        # print(f"In match: {i.home_team} vs {i.away_team}")
        # print(f"Home winning odds: {i.home_win}")
        # print(f"Draw odds: {i.draw}")
        # print(f"Away winning odds: {i.away_win}")
        # print(f"Team {i.victory} won!")
        odds = [m.home_win, m.draw, m.away_win]

        print(f"W meczu {m.home_team} vs {m.away_team} / Zwycięzca: {m.victory} {m.home_odds} {m.draw_odds} {m.away_odds}")
        if max(odds) == m.home_win and max(odds) >= base_percentage and m.home_team == m.victory:
            profit += (m.home_odds * (bet - (bet * tax)) - bet)
            print(profit)
        elif max(odds) == m.away_win and max(odds) >= base_percentage and m.away_team == m.victory:
            profit += (m.away_odds * (bet - (bet * tax)) - bet)
            print(profit)
        elif max(odds) == m.home_win and max(odds) < base_percentage and (m.victory == "Draw" or m.home_team == m.victory):
            profit += (draw_win_odds * (bet - (bet * tax)) - bet)
            print(profit)
        elif max(odds) == m.away_win and max(odds) < base_percentage and (m.victory == "Draw" or m.away_team != m.victory):
            profit += (draw_win_odds * (bet - (bet * tax)) - bet)
            print(profit)
        else:
            profit -= 5
            print(profit)
    return profit

In [138]:
print(count_profit())

W meczu Switzerland vs Cameroon / Zwycięzca: Switzerland 1.82 3.55 5.0
3.008000000000001
W meczu Belgium vs Canada / Zwycięzca: Belgium 1.58 4.4 6.0
4.960000000000002
W meczu Spain vs Costa Rica / Zwycięzca: Spain 1.21 7.8 20.0
5.284000000000002
W meczu Germany vs Japan / Zwycięzca: Japan 1.44 5.0 7.4
0.2840000000000016
W meczu Morocco vs Croatia / Zwycięzca: Draw 3.65 3.3 2.2
1.0040000000000022
W meczu France vs Australia / Zwycięzca: France 1.26 6.0 14.0
1.5480000000000027
W meczu Mexico vs Poland / Zwycięzca: Draw 2.62 3.12 3.25
2.2680000000000033
W meczu Denmark vs Tunisia / Zwycięzca: Draw 1.63 3.85 6.4
-2.7319999999999967
W meczu Argentina vs Saudi Arabia / Zwycięzca: Saudi Arabia 1.12 10.0 25.0
-7.731999999999997
W meczu United States vs Wales / Zwycięzca: Draw 1.7 3.65 5.0
-7.011999999999996
-7.011999999999996


In [9]:
def pc(odds, bet=5):
    return odds * (bet - bet * 0.12) - bet

print(pc(1.58) + pc(1.21) + pc(0) + pc(0) + pc(1.26) + pc(1.4) + pc(0) +
      pc(0) + pc(1.4) + pc(1.7) + pc(1.39) + pc(2.36) + pc(0) + pc(1.92) +
      pc(1.68) + pc(1.3) + pc(2.13) + pc(1.4) + pc(0) + pc(1.4) + pc(0) +
      pc(1.3) + pc(0) + pc(1.3) + pc(0) + pc(2.01) + pc(0) + pc(0) +
      pc(1.16) + pc(1.3) + pc(1.6))
# print(pc(1.74) + pc(0))

-19.479999999999997
